Using Python to scrape out some HTML that I pulled from a site. I manually scraped it because I once again couldn't get my browser to cooperate with the js-rendered page. This outputs two .csv files - one with all the drugs, and one with only the non-discontinued drugs.


In [ ]:
drugs = open('drugsatfda.txt','r') 
from bs4 import BeautifulSoup
import pandas
drugs_read = BeautifulSoup(drugs, 'lxml')

In [ ]:
tablelist = drugs_read.find_all('tr')
drug_names = []
for row in tablelist:
    molrow = row.find_all('ul')
    for x in molrow:
        asdf = x.find_all('li')
        for line in asdf:
            drugrow = line.get_text()
            drug_names.append(drugrow)
# get only the text 
# - all relevant text seems to be within the <li> tags, but some tr rows have multiple li blocks. 
# additionally, i only want to find the li blocks within the ul. 
# i feel like there's probably a better way to do this

In [ ]:
# Grabbing all drugs. 
active_ing = []
ANDA = []
formulation = []
rx = []
mfg = []
for line in drug_names:
    split_string = line.split("|")
    if len(split_string) > 2:
        active_ing.append(split_string[0])
        ANDA.append(split_string[1])
        formulation.append(split_string[2])
        rx.append(split_string[3])
        mfg.append(split_string[4])
ANDA = [a[-7:] for a in ANDA]    
fdalink = []
for x in ANDA:
    newlink = 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo='+ x
    fdalink.append(newlink)
df = pandas.DataFrame(active_ing, columns =['Active Ingredient'])
df['Link'] = fdalink
df['ANDA']= ANDA
df['Formulation']= formulation
df['Prescription'] = rx
df['Manufacturer'] = mfg
df.to_csv('human_drugs.csv')

In [ ]:
# This chunk looks only at drugs that have not been discontinued
active_ing = []
ANDA = []
formulation = []
rx = []
mfg = []
for line in drug_names:
    split_string = line.split("|")
    if len(split_string) > 2:
        if split_string[3] != " Discontinued ":
            active_ing.append(split_string[0].title())
            ANDA.append(split_string[1])
            formulation.append(split_string[2])
            rx.append(split_string[3])
            mfg.append(split_string[4])
ANDA = [a[-7:] for a in ANDA]    
fdalink = []
for x in ANDA:
    newlink = 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo='+ x
    fdalink.append(newlink)
df = pandas.DataFrame(active_ing, columns =['Active Ingredient'])
df['Link'] = fdalink
df['ANDA']= ANDA
df['Formulation']= formulation
df['Prescription'] = rx
df['Manufacturer'] = mfg
df.to_csv('not_discontinued.csv')